## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
import os
import pathlib

In [2]:
this_dir = pathlib.Path(os.path.abspath(""))

In [3]:
data_dir = this_dir / "data"

In [4]:
import pytz
import glob
import json
import requests
import pandas as pd
from datetime import datetime

/Users/slee/.local/share/virtualenvs/california-coronavirus-scrapers-QTClc1WQ/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Download

Retrieve the page

In [5]:
url = "https://services3.arcgis.com/ibgDyuD2DLBge82s/arcgis/rest/services/Fresno_County_Zip_Data_Summary/FeatureServer/0/query?f=json&where=FZIP%3C%3E%2799999%27%20AND%20COUNT_Cases%3E%3D75%20AND%20FZIP%20IS%20NOT%20NULL&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&groupByFieldsForStatistics=FCITY&orderByFields=value%20desc&outStatistics=%5B%7B%22statisticType%22%3A%22sum%22%2C%22onStatisticField%22%3A%22COUNT_Cases%22%2C%22outStatisticFieldName%22%3A%22value%22%7D%5D&resultType=standard&cacheHint=true"

In [6]:
r = requests.get(url)

In [7]:
data = r.json()

## Parse

In [8]:
dict_list = []

In [9]:
for item in data["features"]:
    d = dict(
        county="Fresno",
        area=item["attributes"]["FCITY"],
        confirmed_cases=item["attributes"]["value"],
    )
    dict_list.append(d)

In [10]:
df = pd.DataFrame(dict_list)

In [11]:
df["area"] = df["area"].str.title()

Get timestamp

In [12]:
date_url = "https://services3.arcgis.com/ibgDyuD2DLBge82s/arcgis/rest/services/Fresno_County_Zip_Data_Summary/FeatureServer/0/?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [13]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [14]:
tz = pytz.timezone("America/Los_Angeles")

In [15]:
latest_date = datetime.fromtimestamp(timestamp / 1000, tz).date()

In [16]:
df["county_date"] = latest_date

## Trim

In [17]:
df["county"] = "Fresno"

In [18]:
df["area"] = df["area"].str.title()

In [19]:
c = df[["county", "area", "county_date", "confirmed_cases"]]

## Vet

In [20]:
len(df)

32

In [21]:
df

,county,area,confirmed_cases,county_date
0,Fresno,Fresno,174772,2022-12-30
1,Fresno,Clovis,29103,2022-12-30
2,Fresno,Sanger,12210,2022-12-30
3,Fresno,Selma,10766,2022-12-30
4,Fresno,Reedley,9023,2022-12-30
5,Fresno,Coalinga,7829,2022-12-30
6,Fresno,Kerman,6128,2022-12-30
7,Fresno,Parlier,5887,2022-12-30
8,Fresno,Mendota,3638,2022-12-30
9,Fresno,Kingsburg,3638,2022-12-30


In [22]:
try:
    assert not len(df) > 32
except AssertionError:
    raise AssertionError("Fresno County's place scraper has additional rows")

In [23]:
try:
    assert not len(df) < 32
except AssertionError:
    raise AssertionError("Fresno County's place scraper is missing rows")

In [24]:
len(df)

32

In [25]:
old = pd.read_csv("data/fresno/2022-06-15.csv")

In [26]:
df[~df.area.isin(old.area)]

,county,area,confirmed_cases,county_date
30,Fresno,Dos Palos,107,2022-12-30
31,Fresno,Dunlap,89,2022-12-30


## Export

Mark the current date

In [27]:
tz = pytz.timezone("America/Los_Angeles")

In [28]:
today = datetime.now(tz).date()

In [29]:
slug = "fresno"

In [30]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [31]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [32]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [33]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [34]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)

In [35]:
df[df.area == "Fresno"]

,county,area,confirmed_cases,county_date,date
2634,Fresno,Fresno,128,NaT,2020-04-14 00:00:00
2618,Fresno,Fresno,148,NaT,2020-04-15 00:00:00
2602,Fresno,Fresno,159,NaT,2020-04-16 00:00:00
2586,Fresno,Fresno,166,NaT,2020-04-17 00:00:00
2570,Fresno,Fresno,189,NaT,2020-04-20 00:00:00
...,...,...,...,...,...
0,Fresno,Fresno,174772,2022-12-30,2023-02-14
0,Fresno,Fresno,174772,2022-12-30,2023-02-15
0,Fresno,Fresno,174772,2022-12-30,2023-02-16
0,Fresno,Fresno,174772,2022-12-30,2023-02-17
